<a href="https://colab.research.google.com/github/inspire007/KaggleCompetitions/blob/main/Natural_Language_Processing_with_Disaster_Tweets/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV

nltk.download('stopwords')
from nltk.corpus import stopwords
words = []
test_words = []

def text_processing(text):
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = text.lower().split()
  ps = PorterStemmer()
  text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
  return ' '.join(text)


data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
y = data['target'].values

for i in range(0, len(data)):
  text = text_processing(data['text'][i])
  words.append(text)

for i in range(0, len(data_test)):
  text = text_processing(data_test['text'][i])
  test_words.append(text)

cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(words).toarray()
X_test = cv.transform(test_words).toarray()

models = {
    'svc': SVC(kernel='rbf'),
    'rfc': RandomForestClassifier(n_estimators=500),
    'nv': GaussianNB(),
}

max_acc = 0
selected = False

for name, model in models.items():
  cvs = cross_val_score(estimator = model, X=X, y=y, cv=5)
  mean = cvs.mean()
  print(f'Model {name} mean accuracy: {mean}')
  if mean > max_acc:
    max_acc = mean
    selected = model

print(f'Selected model based on accuracy: {selected}\n')

model = SVC(kernel='rbf')
model.fit(X,y)
y_pred = model.predict(X_test)
ids = data_test['id'].values

np.savetxt('output.csv', np.column_stack((ids,y_pred)), delimiter=',', comments='', header='id,target', fmt=['%d', '%d'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model svc mean accuracy: 0.7119489768361255
Model rfc mean accuracy: 0.6574354854991749
Model nv mean accuracy: 0.6404901454094597
Selected model based on accuracy: SVC()

